In [25]:
import pandas as pd
import numpy as np
import datetime
import time
import os
import glob

import re
from collections import OrderedDict
import networkx as nx
import pickle

# Imports required for random matrix generation
import scipy.sparse as sparse
import scipy.stats as stats

# QAOA and circuit cutting specific imports
from qiskit import QuantumCircuit, qpy
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import PauliList
from circuit_knitting.cutting import (
    partition_problem,
)
# QP specific imports
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo

import random
from collections import OrderedDict
import networkx as nx
#clustering
from sklearn.cluster import SpectralClustering
from sklearn import metrics
import markov_clustering as mc
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import Birch
from sklearn.cluster import KMeans
from string import ascii_uppercase, ascii_lowercase, ascii_letters

#community detection

import networkx.algorithms.community as nx_comm
import community as community_louvain
from collections import defaultdict


def bfs(q1, max_cluster_size):
    q1_nd = [q[0] for q in sorted(q1.degree, key=lambda x: x[1], reverse=True)]

    q_scidx = {}
    scidx_q = {}
    sc_idx = 0
    visited = []

    # for node in sorted(q1.nodes):
    for node in q1_nd:
        if node not in visited:
            scidx_q.setdefault(sc_idx, [])

            nodes_in_sc = set(list(sum(sorted(list(nx.bfs_tree(q1, source=node, depth_limit=1).edges())), ())))
            # print(f'nodes_in_sc: {nodes_in_sc}')

            for k in nodes_in_sc:
                ## required only if using to find cut wire position*****
                q_scidx.setdefault(k, [])  # q_scidx
                q_scidx[k].append(sc_idx)  # q_scidx
                ## ********

                # print(f'visited: {visited}')
                if (k not in visited) and (len(scidx_q[sc_idx]) < max_cluster_size):
                    scidx_q[sc_idx].append(k)
                    visited.append(k)
            sc_idx += 1

    qsc = OrderedDict(sorted(scidx_q.items()))
    cluster_qubit = {}
    bfs_cluster_labels = []

    for qsc_k, qsc_val in qsc.items():
        for qsc_i in qsc_val:
            cluster_qubit.setdefault(qsc_i, [])
            cluster_qubit[qsc_i].append(qsc_k)
    bfs_cluster_labels = [i for val in (OrderedDict(sorted(cluster_qubit.items()))).values() for i in val]
    return bfs_cluster_labels


def sc(data, n_clusters):
    sc = SpectralClustering(n_clusters, affinity='precomputed', n_init=100)
    sc.fit(data)
    return sc.labels_


def kmeans(data, n_clusters):
    kmeans = KMeans(init="k-means++", n_clusters=n_clusters, n_init=4, random_state=0)
    kmeans.fit(data)
    return kmeans.labels_


def kmeans_pca(data, n_clusters):
    pca = PCA(n_components=n_clusters).fit(data)
    kmeans_pca = KMeans(init=pca.components_, n_clusters=n_clusters, n_init=1)
    kmeans_pca.fit(data)
    return kmeans_pca.labels_


def kmeans_random(data, n_clusters):
    kmeans_rand = KMeans(init="random", n_clusters=n_clusters, n_init=4, random_state=0)
    kmeans_rand.fit(data)
    return kmeans_rand.labels_


def birch(data, n_clusters):
    brc = Birch(n_clusters=n_clusters)
    brc.fit(data)
    birch_labels = brc.predict(data)
    return birch_labels


def agglom(data, n_clusters):
    clustering = AgglomerativeClustering(n_clusters=n_clusters).fit(data)
    return clustering.labels_


def louivan_community(G, max_size):
    import community as community_louvain
    clabels = community_louvain.best_partition(G)

    # Split communities if they are larger than max_cluster_size
    return clabels  # split_large_communities(clabels, max_size)


def girvan_newman_community(G, max_size):
    communities_generator = nx.community.girvan_newman(G)
    top_level_communities = next(communities_generator)
    partition = {node: i for i, community in enumerate(top_level_communities) for node in community}
    return partition  # split_large_communities(clabels, max_size)


def label_propagation_community(G, max_size):
    partition = nx.community.label_propagation_communities(G)
    partition_dict = {node: i for i, community in enumerate(partition) for node in community}
    return partition_dict  # split_large_communities(partition_dict, max_size)


def leading_eigenvector_community(G, max_size):
    partition = nx.community.leading_eigenvector(G)
    partition_dict = {node: i for i, community in enumerate(partition) for node in community}
    return partition_dict  # split_large_communities(partition_dict, max_size)


def walktrap_community(G, max_size):
    import igraph as ig
    g = ig.Graph.from_networkx(G)
    walktrap = g.community_walktrap()
    communities = walktrap.as_clustering()
    partition_dict = {node: i for i, community in enumerate(communities) for node in community}
    return partition_dict  # split_large_communities(partition_dict, max_size)


def infomap_community(G, max_size):
    from infomap import Infomap
    im = Infomap()
    for e in G.edges():
        im.add_link(*e)
    im.run()

    # Correctly accessing the nodes and their community assignment
    partition = {}
    for node in im.tree:
        if node.is_leaf:
            partition[node.node_id] = node.module_id

    return partition  # split_large_communities(clabels, max_size)


def clique_percolation_community(G, max_size, k=2):
    partition = list(nx.community.k_clique_communities(G, k))
    partition_dict = {node: i for i, community in enumerate(partition) for node in community}
    return partition_dict  # split_large_communities(partition_dict, max_size)


def split_large_communities(partition, max_size):
    new_partition = {}
    new_community_id = max(partition.values()) + 1  # Start from the next available community ID

    for community in set(partition.values()):
        nodes_in_community = [node for node, comm in partition.items() if comm == community]

        if len(nodes_in_community) > max_size:
            # Split the community into smaller ones
            for i in range(0, len(nodes_in_community), max_size):
                for node in nodes_in_community[i:i + max_size]:
                    new_partition[node] = new_community_id
                new_community_id += 1
        else:
            # If the community is within the size limit, keep it unchanged
            for node in nodes_in_community:
                new_partition[node] = community

    return new_partition


def sc2(*argv):
    data = argv[0][0]
    n_clusters = argv[0][1]
    sc = SpectralClustering(n_clusters, affinity='precomputed', n_init=100)
    sc.fit(data)
    # TODO: verify sub cluster size<max_cluster_size. Is this still required??
    return sc.labels_


def kmeans2(*argv):
    data = argv[0][0]
    n_clusters = argv[0][1]

    kmeans = KMeans(init="k-means++", n_clusters=n_clusters, n_init=4, random_state=0)
    kmeans.fit(data)
    return kmeans.labels_


def birch2(*argv):
    data = argv[0][0]
    n_clusters = argv[0][1]
    brc = Birch(n_clusters=n_clusters)
    brc.fit(data)
    birch_labels = brc.predict(data)
    return birch_labels


def agglom2(*argv):
    data = argv[0][0]
    n_clusters = argv[0][1]
    clustering = AgglomerativeClustering(n_clusters=n_clusters).fit(data)
    return clustering.labels_

#TODO: try running this
def louivan_community2(*argv):
    G = argv[0][0]
    # print('in louivan')
    # print(G)
    # max_size = argv[0][1]
    # print(max_size)
    import community as community_louvain
    clabels = community_louvain.best_partition(G)


def girvan_newman_community2(*argv):
    G = argv[0][0]
    # max_size = argv[0][1]
    communities_generator = nx.community.girvan_newman(G)
    top_level_communities = next(communities_generator)
    partition = {node: i for i, community in enumerate(top_level_communities) for node in community}
    return partition  # split_large_communities(clabels, max_size)


def label_propagation_community2(*argv):
    G = argv[0][0]
    # max_size = argv[0][1]
    partition = nx.community.label_propagation_communities(G)
    partition_dict = {node: i for i, community in enumerate(partition) for node in community}
    return partition_dict  # split_large_communities(partition_dict, max_size)


def leading_eigenvector_community2(*argv):
    G = argv[0][0]
    # max_size = argv[0][1]
    partition = nx.community.leading_eigenvector(G)
    partition_dict = {node: i for i, community in enumerate(partition) for node in community}
    return partition_dict  # split_large_communities(partition_dict, max_size)


def walktrap_community2(*argv):
    G = argv[0][0]
    # max_size = argv[0][1]

    import igraph as ig
    g = ig.Graph.from_networkx(G)
    walktrap = g.community_walktrap()
    communities = walktrap.as_clustering()
    partition_dict = {node: i for i, community in enumerate(communities) for node in community}
    return partition_dict  # split_large_communities(partition_dict, max_size)


def infomap_community2(*argv):
    G = argv[0][0]
    # max_size = argv[0][1]

    from infomap import Infomap
    im = Infomap()
    for e in G.edges():
        im.add_link(*e)
    im.run()

    # Correctly accessing the nodes and their community assignment
    partition = {}
    for node in im.tree:
        if node.is_leaf:
            partition[node.node_id] = node.module_id

    return partition  # split_large_communities(clabels, max_size)


##TODO: Rest of the partition methods to be updated with *args

# from circuit_knitting_toolbox.circuit_cutting.wire_cutting import cut_circuit_wires

# useful additional packages
#from qiskit.tools.visualization import plot_histogram
#from qiskit.algorithms.minimum_eigensolvers import SamplingVQE, NumPyMinimumEigensolver
#from qiskit.algorithms.optimizers import SPSA
#from qiskit.utils import algorithm_globals

latin_start = 0x0100  # Start of Latin Extended-A block
latin_end = latin_start + 1000  # 1000 characters from the start

labels = ''.join(chr(i) for i in range(latin_start, latin_end))
#labels = ascii_uppercase + ascii_lowercase   #string.ascii_letters#`
labels

'''
Methods required to generate random ER graphs
'''


# Erdos Renyi graph
def generate_er_graph(n, p, seed):
    G = nx.erdos_renyi_graph(n, p, seed=seed)
    M = nx.adjacency_matrix(G).todense()
    return M, G


'''
Methods required to generate random sparse matrices
'''


def sprandsym(n, density, seed):
    np.random.seed((seed))
    rvs = stats.poisson(25, loc=10).rvs
    X = sparse.random(n, n, density=density, data_rvs=rvs)
    upper_X = sparse.triu(X)
    result = upper_X + upper_X.T - sparse.diags(X.diagonal())
    return result


def binarize_sparse_matrix(sparse_matrix):
    # create a copy of the sparse matrix to keep the operation non-destructive
    sparse_copy = sparse_matrix.copy()
    # sparse_copy=sparse_copy-sparse.diags(sparse_copy.diagonal())
    # find the coordinates of non-zero elements
    non_zero_coords = sparse_copy.nonzero()
    # set those elements to 1
    sparse_copy[non_zero_coords] = 1
    return sparse_copy


def generate_graph_from_matrix(binarized_sparse_mat):
    G = nx.from_scipy_sparse_array(binarized_sparse_mat)
    return G


# create the quadratic program instance and define the variables
def create_qp_from_qmatrix(Q_matrix):
    max_keys = Q_matrix.shape[0]
    qp = QuadraticProgram('QUBO Matrix Optimization')
    x = qp.binary_var_list(name='x', keys=range(1, max_keys + 1))

    linear_vars = {qp.get_variable(i).name: Q_matrix[i, j]
                   for i in range(max_keys) for j in range(max_keys) if i == j}
    quadratic_vars = {(qp.get_variable(i).name, qp.get_variable(j).name): Q_matrix[i, j]
                      for i in range(max_keys) for j in range(max_keys) if i != j}

    qp.minimize(linear=linear_vars, quadratic=quadratic_vars)
    return qp
    # print(self.qp.prettyprint())


def create_qaoa_ansatz(qp):
    # self.create_qp_from_qmatrix()
    h_qubo, offset = qp.to_ising()
    # print(h_qubo)
    qaoa_ansatz = QAOAAnsatz(cost_operator=h_qubo, reps=1, )
    qaoa_ansatz.entanglement = 'linear'
    params = len(qaoa_ansatz.parameters)
    theta_range = np.linspace(0, np.pi, params)
    qaoa_qc = qaoa_ansatz.bind_parameters(theta_range)
    decomposed_qaoa_ansatz = qaoa_qc.decompose().decompose().decompose().decompose()
    return h_qubo, offset, decomposed_qaoa_ansatz


def get_subgraph_properties1(G):
    cnt = 0
    subgraph = (G.subgraph(c) for c in nx.connected_components(G))
    subgraph_prop = {}
    prop = []
    max_size = []
    max_subgraph_nodes = ''
    for s in subgraph:
        # print(s.nodes())
        n = tuple(s.nodes())
        subgraph_prop[n] = nx.adjacency_matrix(s).todense()
        # print(s.size())
        # print(f'Subgraph {cnt}:: Num of Edges: {s.size()},  Nodes : {s.nodes()}  ')
        cnt += 1
        max_size.append(len(s.nodes()))
        if len(s.nodes) == np.max(max_size):
            max_subgraph_nodes = s.nodes()

    # print(max_subgraph_nodes)
    return cnt, np.max(max_size), subgraph_prop, max_subgraph_nodes


def ckt(qaoa_decomposed, part_lbl, observables):
    ordered_part_lbl = OrderedDict(sorted(part_lbl.items()))
    partition_labels = ''.join(ordered_part_lbl.values())
    #print(f'\nPartition labels for CKT: {partition_labels}')
    print(f'CKT labels: {partition_labels}')
   
    start_time = time.time()

    partitioned_problem = partition_problem(circuit=qaoa_decomposed,
                                            partition_labels=partition_labels,
                                            observables=observables)
    ckt_runtime = time.time() - start_time
    bases = partitioned_problem.bases
    sampling_overhead = np.prod([basis.overhead for basis in bases])
    print(f"SO: {sampling_overhead}")
    return partition_labels, sampling_overhead, ckt_runtime

def sc(data, n_clusters):
    sc = SpectralClustering(n_clusters, affinity='precomputed', n_init=100)
    sc.fit(data)
    return sc.labels_

def partitioning2(max_cluster_size, qsubgraph_prop, partition_method):
    # cm_part_lbl = {}
    # partitioning_time = {}
    start_time = time.time()
    part_lbl = {}
    max_key_cnt = -1
    clabels = ''
    for i, key in enumerate(qsubgraph_prop.keys()):
        # print(f'Subgraph nodes : {key}')
        if len(key) > max_cluster_size:
            data = qsubgraph_prop[key]
            n_clusters = int(np.ceil(len(key) / max_cluster_size))
            q1 = nx.from_numpy_array(data)

            '''if 'spectral' in partition_method:
                clabels = sc(data, n_clusters)#, key, max_cluster_size)'''
            algos = {
            
                'spectral-clustering':
                    {
                        'func': sc2,
                        'params': (data, n_clusters)
                    },
                'kmeans':
                    {
                        'func': kmeans2,
                        'params': (data, n_clusters)
                    },
                # 'kmeans-pca' : { 'func':kmeans_pca2, 'params':(data, n_clusters) },
                # 'kmeans-random' : {'func':kmeans_random2, 'params':(data, n_clusters)},
                'birch':
                    {
                        'func': birch2,
                        'params': (data, n_clusters)
                    },
                'agglom':
                    {
                        'func': agglom2,
                        'params': (data, n_clusters)
                    },
                'louivan':
                    {
                        'func': louivan_community2,
                        'params': (q1, max_cluster_size)},
                'girvan-newman':
                    {
                        'func': girvan_newman_community2,
                        'params': (q1, max_cluster_size)
                    },
                'label-propagation':
                    {
                        'func': label_propagation_community2,
                        'params': (q1, max_cluster_size)},
                'leading-eigenvector':
                    {
                        'func': leading_eigenvector_community2,
                        'params': (q1, max_cluster_size)},
                'walktrap':
                    {
                        'func': walktrap_community2,
                        'params': (q1, max_cluster_size)},
                'infomap':
                    {
                        'func': infomap_community2,
                        'params': (q1, max_cluster_size)},
                # 'clique-percolation' : {'func':clique_percolation_community2, 'params':(q1,max_cluster_size)}
            }

            # pm = ops[partition_method](func_arg[partition_method])
            func = algos[partition_method]['func']
            params = algos[partition_method]['params']

            # call the partition method
            clabels = func(params)

            # increment the sclbl with max_key_cnt so that the next sub-graphs labels are not repeated
            cluster_lbls = [lbl + max_key_cnt + 1 for lbl in clabels]
            for j, k_ in enumerate(key):
                if cluster_lbls[j] < len(labels):
                    part_lbl[k_] = labels[cluster_lbls[j]]
                else:
                    print(
                        f'Error: Index out of range. cluster_lbls[j]: {cluster_lbls[j]}, Length of labels: {len(labels)}')

            max_key_cnt = np.max(cluster_lbls)

        else:
            # Handle smaller subgraphs
            max_key_cnt += 1
            for k in key:
                if max_key_cnt < len(labels):
                    part_lbl[k] = labels[max_key_cnt]
                else:
                    print(f'Error: Index out of range. max_key_cnt: {max_key_cnt}, Length of labels: {len(labels)}')
        # print('done part')
        # cm_part_lbl[cm] = (part_lbl, partitioning_runtime)
        # print(f'Partition labels for method {cm}: {part_lbl}')
    partitioning_time = time.time() - start_time

    return part_lbl, partitioning_time


base_file_path = 'C:\\Users\\Administrator\\PycharmProjects\\cc\\'
qaoa_graph_obj_dir = os.path.join(base_file_path,'qaoa_graph_objects/')
qaoa_output = os.path.join(base_file_path,'qaoa_output/')
ckt_output = os.path.join(base_file_path,'ckt_output/spect/jun22')


def ckt_build_qaoa(mat_size, n_times_p, random_seeds, matrix_type):
    '''
    ## Create sparse matrix for a given size and density
    ## Convert the sparse matrix into QUBO, map Ising hamiltonian
    ## Generate QAOA ansatz for QUBO and assign its observables
    '''

    cols = ['n', 'np', 'seed','File Path','Graph File','qaoa Ansatz File',
            'Ising Hamiltonian Runtime', 'Ansatz Building Runtime']

    df = pd.DataFrame(columns=cols)
    # df['Graph Prop'] = df['Graph Prop'].astype('object')
    # df['qubitOp'] = df['qubitOp'].astype('object')

    i = 0
    for n in mat_size:
        for p in n_times_p:
            for seed in random_seeds:
                # multi-thread code to go here for a set of seeds??
                try:
                    print(f'\n\nQAOA for size {n}, n*p {p}, seed {seed}')
                    if i == 0:
                        seed1 = seed
                    i += 1
                    ## Create sparse matrix for a given size and density
                    # record the matrix creation time
                    start_time = time.time()
                    if 'random_sparse' in matrix_type:
                        M = sprandsym(n, p, seed)
                        M = binarize_sparse_matrix(M)
                        q = generate_graph_from_matrix(M)

                    ## Get adjacency matrix for a random Erdos Renyi Graph
                    elif 'er_graph' in matrix_type:
                        M, q = generate_er_graph(n, p, seed)
                    matrix_creation_time = time.time() - start_time

                    # save graph object to file
                    # graph_filename=os.path.join(qaoa_graph_obj_dir,f'{n}_{p}_{seed}_graph.pickle')
                    graph_filename = f'{n}_{p}_{seed}_graph.pickle'
                    pickle.dump(q, open(os.path.join(qaoa_graph_obj_dir, graph_filename), 'wb'))

                    # Get subgraphs' properties
                    # qnum_sub_graphs, largest_subgraph_size, qsubgraph_prop, max_subgraph_nodes = get_subgraph_properties1(q)

                    ## Convert the sparse matrix into QUBO
                    # Timing for QUBO conversion
                    start_time = time.time()
                    qp = create_qp_from_qmatrix(M)
                    # here
                    qp2qubo = QuadraticProgramToQubo()
                    qubo = qp2qubo.convert(qp)
                    qubitOp, offset = qubo.to_ising()
                    qubo_conversion_time = time.time() - start_time
                    #print(qubitOp)

                    ## Generate and save QAOA ansatz for QUBO
                    # Timing for QAOA
                    start_time = time.time()
                    qaoa = QAOAAnsatz(cost_operator=qubitOp, reps=1)
                    qaoa_observable_pat = '[A-Z]+'
                    #observables = PauliList(re.findall(qaoa_observable_pat, str(qubitOp)))
                    observables = (PauliList(re.findall(r"'([A-Z]+)'", str(qubitOp))))
                    qaoa_time = time.time() - start_time

                    #qa = qaoa.decompose().decompose().decompose().decompose()
                    # display(qa.draw(scale=0.4))

                    # pickle the qaoa ansatz with qpy- problematic when n>250
                    qc_file = os.path.join(qaoa_graph_obj_dir,f'{n}_{p}_{seed}_qaoa.qpy')
                    ''' with open(file=qc_file, mode='wb') as qcfile:
                        qpy.dump(qaoa, qcfile)'''

                    #pickle the qaoa ansatz
                    qc_file = os.path.join(qaoa_graph_obj_dir,f'{n}_{p}_{seed}_qaoa.pickle')
                    qc_file = f'{n}_{p}_{seed}_qaoa.pickle'

                    with open(file=os.path.join(qaoa_graph_obj_dir, qc_file), mode='wb') as qcfile:
                        pickle.dump(qaoa, qcfile)

                    # Populate the df
                    df.loc[i, 'n'] = n
                    df.loc[i, 'np'] = p
                    df.loc[i, 'seed'] = seed
                    df.loc[i, 'File Path'] = qaoa_graph_obj_dir
                    df.loc[i, 'Graph File'] = graph_filename
                    df.loc[i, 'qaoa Ansatz File'] = qc_file
                    # df.loc[i,'qubitOp'] = str(qubitOp)
                    # df.loc[i,'QP'] = np.matrix(M)
                    df.loc[i, 'Ising Hamiltonian Runtime'] = np.round(qubo_conversion_time, 5)
                    df.loc[i, 'Ansatz Building Runtime'] = np.round(qaoa_time, 5)

                except Exception as e:
                    print(f"An error occurred: {e}")
                    # Optionally, save the DataFrame
                    current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f'failed_exp/exp_data_up_to_failure_{current_time}.csv'
                    df.to_csv(filename, index=False)
                    # re-raise the exception if you want to stop the loop
                    raise e
    current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M")

    filename = f'exp_' + str(n) + '_' + str(p) + '_' + str(seed1) + '_' + str(seed) + '_qaoa_' + current_time + '.csv'
    df.to_csv(os.path.join(qaoa_output, filename), index=False)
    return filename


def ckt_cut_qaoa(exp_data_file, n_max_qubit_dict, max_exp_each_n, partition_methods, theta, start_seed=None, end_seed=None):
    cols = ['n', 'np', 'seed', 'Max Qubit Count',
            'Partition Method', 'Partition Runtime', 'CKT Runtime']  # 'qaoa Ansatz File'
    failed_cols = ['n', 'np', 'seed', 'Max Qubit Count', 'Partition Method']

    all_filenames = []

    df = pd.DataFrame(columns=cols)
    df_failed = pd.DataFrame(columns=failed_cols)
    i = 0
    #_mat_size = n_max_qubit_dict
    #_q_cnt = 
    #for max_qubit_cnt in max_qubit_counts:
    for file in exp_data_file:
        try:
           # print(f'exp#: {i}')
            print(file)

            input_df = pd.read_csv(file)
            # print(input_df.dtypes)
            if start_seed is None:
                start_seed = 0
            if end_seed is None:
                end_seed = 1000000
            #input_df = input_df[(input_df['seed'] >= start_seed) & (input_df['seed'] <= end_seed)]
            print(f"input seed values: {input_df['seed'].values}")
            #_n = 0
            #_n_cnt = 0
            for i, row in enumerate(input_df.itertuples(index=False), 1):
                    print(f'exp #: {i}')
                    n = row[0]
                    #temp_n = _n
                    #if (_n_cnt != max_exp_each_n):
                    #    if (n != temp_n)  :
                    #       _n = n
                    #        _n_cnt += 1
    
                    #   if _n_cnt >= max_exp_each_n:
                    #       print(f'max exp done for {n}')
                    #        continue
                        
                    
                    p = row[1]
                    seed = row[2]
                    seed1 = seed

                    if True: # (seed=='96542833605421401283018193477403') and (p==5.0): #and (n==20)

                        #run only the required files???
                    
                        if n not in n_max_qubit_dict.keys():
                            print(f'No circuits to cut for n={n}')
                        
                        else:
                            max_qubit_counts = n_max_qubit_dict[n]
                            _max_qcnt = [int(np.ceil(mqc)) for mqc in max_qubit_counts]
                            for max_qubit_cnt in _max_qcnt:
                                # temp: skip the cases where no. of cuts = 0 and assign SO 1.0
                                if max_qubit_cnt >= n:
                                    seed1 = seed
                                    print('max qubit >= n')
                                    df.loc[i, 'n'] = n
                                    df.loc[i, 'np'] = p
                                    df.loc[i, 'seed'] = seed
                                    df.loc[i, 'Max Qubit Count'] = max_qubit_cnt
                                    df.loc[i, 'Partition Method'] = 'spect'
                                    df.loc[i, 'Partition Runtime'] = '0.0'
                                    df.loc[i, 'CKT Runtime'] = '0.0'
                                    df.loc[i, 'Sampling Overhead'] = 1.0
                                    df.loc[i, 'Circuit Reload Time'] = '0.0'
                                    current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M")
                                    filename = f'exp_' + str(n) + '_' + str(p) + '_' + str(max_qubit_cnt) + '_' + str(
                                        start_seed) + '_' + str(end_seed) + '_cktCut_' + current_time + '.csv'
                                    #print(filename)
                                    df.to_csv(os.path.join(ckt_output, filename), index=False)
                                    all_filenames.append(filename)
                                    continue
            
                                if i == 1:
                                    seed1 = seed
                                    # seedn = seed if i==input_df.shape[0]
            
                                file_path = row[3]
                                _file_path = os.path.join(base_file_path, file_path)
                                graph_filename = os.path.join(_file_path, row[4])
                                print(f'graph: {graph_filename}')
            
                                # import ast
                                # qsubgraph_prop = eval(qsub_prop)
            
                                # load graph object from file
                                q = pickle.load(open(graph_filename, 'rb'))
                                #print('loaded q')
                                qnum_sub_graphs, largest_subgraph_size, qsubgraph_prop, max_subgraph_nodes = get_subgraph_properties1(
                                    q)
            
                                qc_file = os.path.join(_file_path, row[5])
                                # qubitOp = row[7]
                                #"p0-["
                                # M = row[7]
                                # print(M)
                                # print(np.matrix(M))
                                M = nx.adjacency_matrix(q).todense()
                                qp = create_qp_from_qmatrix(M)
                                qp2qubo = QuadraticProgramToQubo()
                                qubo = qp2qubo.convert(qp)
                                qubitOp, offset = qubo.to_ising()
                                # read the file with qaoa ansatz object
                                st = time.time()
            
                                # qpy problematic with n>250
                                '''with open(qc_file, mode='rb') as qc:
                                    reloaded_qaoa_ansatz = qpy.load(qc)[0]'''
            
                                reloaded_qaoa_ansatz = pickle.load(open(qc_file, 'rb'))
                                qaoa_decomposed = reloaded_qaoa_ansatz.decompose().decompose().decompose()#.decompose()
                                # display(qaoa_decomposed.draw(scale=0.4))
                                reload_time = time.time() - st
                                print(f'qaoa reload time: {reload_time}')
                                params = len(qaoa_decomposed.parameters)
                                # Bind parameters
                                #theta_range = np.linspace(0, np.pi / 2, params)
                                #print('before theta')
                                theta_range = np.linspace(theta[0], theta[1], params)  # 0, params)#
                                #print('after theta')
                                #qaoa_qc = qaoa_decomposed.assign_parameters(theta_range)
                                qaoa_qc = qaoa_decomposed.assign_parameters(theta_range)
                                qaoa_observable_pat = '[A-Z]+'
                                #observables = PauliList(re.findall(qaoa_observable_pat, str(qubitOp)))
                                observables = (PauliList(re.findall(r"'([A-Z]+)'", str(qubitOp))))
                                # print(qubitOp)
                                # Get subgraphs' properties
                                for pm_ in partition_methods:
                                    print(f'\nCutting for {n}, np {p}, qcnt {max_qubit_cnt}, seed {seed}, {pm_}')
                                
                                    try:
                                       # if seed1 is np.empty():
                                        #seed1 = seed
                                        #print(pm_)
                                        #i += 1
                                        pm_part_lbl, partitioning_runtime = partitioning2(max_qubit_cnt,
                                                                                          qsubgraph_prop,
                                                                                          pm_)
                                        #print(pm_part_lbl)
            
                                        sc_partlabel, sampling_overhead, ckt_runtime = ckt(qaoa_qc,
                                                                                           pm_part_lbl,
                                                                                           observables)
                                        df.loc[i, 'n'] = n
                                        df.loc[i, 'np'] = p
                                        df.loc[i, 'seed'] = seed
                                        df.loc[i, 'Max Qubit Count'] = max_qubit_cnt
                                        df.loc[i, 'Partition Method'] = pm_
                                        df.loc[i, 'Partition Runtime'] = partitioning_runtime
                                        df.loc[i, 'CKT Runtime'] = ckt_runtime
                                        df.loc[i, 'Sampling Overhead'] = sampling_overhead
                                        df.loc[i, 'Circuit Reload Time'] = reload_time
                                        i += 1
                                    except Exception as e:
                                        print(f"An error occurred: {e}")
                                        # Optionally, save the DataFrame
                                        current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M")
                                        filename = f'exp_' + str(n) + '_' + str(p) + '_' + str(start_seed) + str(
                                            end_seed) + '_cktCut_upToFailure_' + current_time + '.csv'
                                        # filename = f'exp_cktCutQaoa_up_to_failure_{current_time}.csv'
                                        #df.to_csv(os.path.join(ckt_output, filename), index=False)
            
                                        # save the failed metadata
                                        df_failed.loc[i, 'n'] = n
                                        df_failed.loc[i, 'np'] = p
                                        df_failed.loc[i, 'seed'] = seed
                                        df_failed.loc[i, 'Max Qubit Count'] = max_qubit_cnt
                                        df_failed.loc[i, 'Partition Method'] = pm_
                                        continue
            
                                #current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M")
                                #filename = f'exp_' + str(n) + '_' + str(p) + '_' + str(max_qubit_cnt) + '_' + str(
                                #    seed1) + '_' + str(seed) + '_cktCut_' + current_time + '.csv'
                                #print(filename)
                                #df.to_csv(os.path.join(ckt_output, filename), index=False)
                                #all_filenames.append(filename)

                    if not df_failed.empty:
                        failed_filename = f'FailedExp_' + str(n) + '_' + str(p) + '_' + str(start_seed) + str(
                                        end_seed) + '_cktCut_' + current_time + '.csv'
                        #df_failed.to_csv(os.path.join(ckt_output, failed_filename), index=False)
        except Exception as e:
                print(f"An error occurred: {e}")
                current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M")
                filename = f'exp_' + str(n) + '_' + str(p) + '_' + str(start_seed) + str(
                                        end_seed) + '_cktCut_upToFailure_' + current_time + '.csv'
                df.to_csv(os.path.join(ckt_output, filename), index=False)
        current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M")
        filename = f'exp_' + str(n) + '_' + str(p) + '_' + str(max_qubit_cnt) + '_' + str(
            seed1) + '_' + str(seed) + '_cktCut_' + current_time + '.csv'
        #print(filename)
        df.to_csv(os.path.join(ckt_output, filename), index=False)
        all_filenames.append(filename)
    return all_filenames




In [19]:
#TODO:
#1. pick a small range matrix - change the ckt parameters - get the baseline for the optimal solution  - use the new tool
#2. re-run the exp with max qubit size [I think for max_circuit_size we should set limits at [n/4,n/2,3n/4, 65 and 127] where n is matrix size, 
    #while making sure max_circuit_size<n. Then we run experiments for matrix sizes from 10-100 as we have now.]   for all methods
#3. hamlib

#### Build Circuits

In [20]:
##Build circuits
#matrix_sizes=[6]#[20, 30, 40, 50, 60, 70, 80, 90, 100]

##n*p = 1
#np_ =[0.35]#[0.05, 0.06, 0.07, 0.09, 0.1, 0.2, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.05, 1.1, 1.15]# [2, 2.5, 3, 3.5, 3.6]#[0.95, 1, 1.05, 1.1,1.15]#[

#matrix_densities = np_ #[0.2]
#num_of_experiments = 1

#random_seeds = sorted([random.randint(10, 100000) for _ in range(num_of_experiments)])
#ckt_build_qaoa(matrix_sizes, matrix_densities, random_seeds, 'er_graph')


#### Cut circuits

In [21]:
np_dict = [{n: [n/4, n/2, 3*n/4, 65, 127]} for n in [20, 30, 40, 50, 60, 70, 80, 90, 100] ]
np_dict


[{20: [5.0, 10.0, 15.0, 65, 127]},
 {30: [7.5, 15.0, 22.5, 65, 127]},
 {40: [10.0, 20.0, 30.0, 65, 127]},
 {50: [12.5, 25.0, 37.5, 65, 127]},
 {60: [15.0, 30.0, 45.0, 65, 127]},
 {70: [17.5, 35.0, 52.5, 65, 127]},
 {80: [20.0, 40.0, 60.0, 65, 127]},
 {90: [22.5, 45.0, 67.5, 65, 127]},
 {100: [25.0, 50.0, 75.0, 65, 127]}]

In [22]:
int(np.ceil(23.0))

23

In [23]:
'''np_dict = {
    20:[5,10],
    30:[5,10,20],
    40:[5,10,20,30],
    50:[10,20,30,40],
    60:[20,30,40,50],
    70:[30,40,50],
    80:[30,40,50],
    90:[40,50,80],
    100:[50,80,90]

}'''

np_dict = {
    20: [5.0, 10.0, 15.0],
     30: [7.5, 15.0, 23],
     40: [10.0, 20.0, 30],
     50: [12.5, 25.0, 38],
     60: [15.0, 30.0, 45],
     70: [17.5, 35.0, 53, 65],
     80: [20.0, 40.0, 60, 65],
     90: [22.5, 45.0, 68],
     100: [25.0, 50.0, 75.0, 65]
}
np_ = [0.05, 0.09, 0.2, 0.4, 0.7, 0.8, 0.9, 1, 1.05, 1.1, 1.15, 1.25, 1.35, 1.45, 1.75, 2, 2.5, 3, 4, 5]

#np_ = [0.05, 0.06, 0.07, 0.09, 0.1, 0.2, 0.4, 0.5, 0.7, 0.8, 0.9, 1, 1.05, 1.1, 1.15]

#files ='exp_100_1.15_57801_832134_qaoa_20240618_1119.csv'# 'exp_50_0.5_9935_9935*.csv' #'exp_' + str(n1) + '_' + str(n2) +  '_' + str(p1) + '_' + str(p2) + '_9999'+ '_*.csv'
files='exp_100_5_3160689083137057833799141206129_96542833605421401283018193477403_qaoa_20240625_0822.csv'
qaoa_out = os.path.join(base_file_path, 'qaoa_output')
qaoa_files = glob.glob(os.path.join(qaoa_out, files))
print(qaoa_files)
start_seed =100000000000 
end_seed = 100000000000000000000000000000000



['C:\\Users\\Administrator\\PycharmProjects\\cc\\qaoa_output\\exp_100_5_3160689083137057833799141206129_96542833605421401283018193477403_qaoa_20240625_0822.csv']


In [ ]:
st = time.time()
ckt_cut_qaoa(qaoa_files, np_dict, 2, ['birch'], [0, np.pi/2],start_seed=start_seed, end_seed=end_seed) #, 'birch', 'agglom']
#exp_data_file, n_max_qubit_dict, max_exp_each_n, partition_methods, theta, start_seed=None, end_seed=None

print(time.time()-st)

C:\Users\Administrator\PycharmProjects\cc\qaoa_output\exp_100_5_3160689083137057833799141206129_96542833605421401283018193477403_qaoa_20240625_0822.csv
input seed values: ['3160689083137057833799141206129' '16614821052023653086125013153459'
 '19462302592384759755737891036649' ... '81140444296104640062273464805578'
 '94154307026748860642691426938873' '96542833605421401283018193477403']
exp #: 1
graph: C:\Users\Administrator\PycharmProjects\cc\qaoa_graph_objects/jun18_np\20_0.05_3160689083137057833799141206129_graph.pickle
qaoa reload time: 0.2021932601928711

Cutting for 20, np 0.05, qcnt 5, seed 3160689083137057833799141206129, birch
CKT labels: ĀāĂăĄąĆćĈĉĊċČčĎĄĂďĐđ
SO: 1.0
graph: C:\Users\Administrator\PycharmProjects\cc\qaoa_graph_objects/jun18_np\20_0.05_3160689083137057833799141206129_graph.pickle
qaoa reload time: 0.14973688125610352

Cutting for 20, np 0.05, qcnt 10, seed 3160689083137057833799141206129, birch
CKT labels: ĀāĂăĄąĆćĈĉĊċČčĎĄĂďĐđ
SO: 1.0
graph: C:\Users\Administrator

C:\ProgramData\anaconda3\envs\CC\Lib\site-packages\scipy\cluster\hierarchy.py:796: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  return linkage(y, method='ward', metric='euclidean')


qaoa reload time: 0.3240222930908203

Cutting for 20, np 1.25, qcnt 10, seed 3160689083137057833799141206129, birch
CKT labels: ĀĀāĀĀĂĂĀăāĂāĄāāĀāąĀā
SO: 1.0
graph: C:\Users\Administrator\PycharmProjects\cc\qaoa_graph_objects/jun18_np\20_1.25_3160689083137057833799141206129_graph.pickle
qaoa reload time: 0.31873488426208496

Cutting for 20, np 1.25, qcnt 15, seed 3160689083137057833799141206129, birch
CKT labels: ĀĀāĀĀĂĂĀăāĂāĄāāĀāąĀā
SO: 1.0
exp #: 222
graph: C:\Users\Administrator\PycharmProjects\cc\qaoa_graph_objects/jun18_np\20_1.25_16614821052023653086125013153459_graph.pickle
qaoa reload time: 0.6396257877349854

Cutting for 20, np 1.25, qcnt 5, seed 16614821052023653086125013153459, birch
CKT labels: ĂĀĄąăāăāăĀāĂāĂĀĆāāāć
SO: 205891132094650.25
graph: C:\Users\Administrator\PycharmProjects\cc\qaoa_graph_objects/jun18_np\20_1.25_16614821052023653086125013153459_graph.pickle
qaoa reload time: 0.29679179191589355

Cutting for 20, np 1.25, qcnt 10, seed 16614821052023653086125013153459

C:\ProgramData\anaconda3\envs\CC\Lib\site-packages\scipy\cluster\hierarchy.py:796: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  return linkage(y, method='ward', metric='euclidean')


qaoa reload time: 0.6295549869537354

Cutting for 20, np 1.25, qcnt 10, seed 78603990680732601915491443719660, birch
CKT labels: ĀĂăāĄăāăĂĂĂāąĂāāĂāĂĂ
SO: 81.00000000000007
graph: C:\Users\Administrator\PycharmProjects\cc\qaoa_graph_objects/jun18_np\20_1.25_78603990680732601915491443719660_graph.pickle


...

qaoa reload time: 3.5942695140838623

Cutting for 50, np 5.0, qcnt 25, seed 94154307026748860642691426938873, birch
CKT labels: ĀĀĀĀĀĀĀāĀĀāĀĀĀĀĀĀĀĀĀĀāĀĀĀĀĀĀĀĀĀĀĀĀĀĀĀāĀĀĀāĀĀĀĀĀĀĀĀ
SO: 3.815204244769506e+29
graph: C:\Users\Administrator\PycharmProjects\cc\qaoa_graph_objects/jun18_np\50_5_94154307026748860642691426938873_graph.pickle


C:\ProgramData\anaconda3\envs\CC\Lib\site-packages\scipy\cluster\hierarchy.py:796: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  return linkage(y, method='ward', metric='euclidean')


qaoa reload time: 3.176955461502075

Cutting for 50, np 5.0, qcnt 38, seed 94154307026748860642691426938873, birch
CKT labels: ĀĀĀĀĀĀĀāĀĀāĀĀĀĀĀĀĀĀĀĀāĀĀĀĀĀĀĀĀĀĀĀĀĀĀĀāĀĀĀāĀĀĀĀĀĀĀĀ
SO: 3.815204244769506e+29
exp #: 1600
graph: C:\Users\Administrator\PycharmProjects\cc\qaoa_graph_objects/jun18_np\50_5_96542833605421401283018193477403_graph.pickle


C:\ProgramData\anaconda3\envs\CC\Lib\site-packages\scipy\cluster\hierarchy.py:796: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  return linkage(y, method='ward', metric='euclidean')


qaoa reload time: 2.85244083404541

Cutting for 50, np 5.0, qcnt 13, seed 96542833605421401283018193477403, birch
CKT labels: ĀāĀĀĀĀĀĀĂĀĂāĀăĀĀĄāĀĀĀĀĀĀĀāĀāĀāĀĀāăĀĀĀĀĀĀĀāĀĀĀĀĀĀĀĀ
SO: 1.3302794647291351e+39
graph: C:\Users\Administrator\PycharmProjects\cc\qaoa_graph_objects/jun18_np\50_5_96542833605421401283018193477403_graph.pickle
qaoa reload time: 2.7500016689300537

Cutting for 50, np 5.0, qcnt 25, seed 96542833605421401283018193477403, birch
CKT labels: āĀāāāāāāĀāĀĀāĀāāĂĀāāāāāāāĀāĀāĀāāĀĀāāāāāāāĀāāāāāāāā
SO: 5.233476330273664e+26
graph: C:\Users\Administrator\PycharmProjects\cc\qaoa_graph_objects/jun18_np\50_5_96542833605421401283018193477403_graph.pickle
qaoa reload time: 2.8437986373901367

Cutting for 50, np 5.0, qcnt 38, seed 96542833605421401283018193477403, birch
CKT labels: āĀāāāāāāĀāĀĀāĀāāĂĀāāāāāāāĀāĀāĀāāĀĀāāāāāāāĀāāāāāāāā
SO: 5.233476330273664e+26
exp #: 1601
graph: C:\Users\Administrator\PycharmProjects\cc\qaoa_graph_objects/jun18_np\60_0.05_3160689083137057833799141206129_g

In [11]:
np_dict.keys()

dict_keys([40])